### Introduction

This notebook prepares the match metadata used in my thesis on toxic behavior in Dutch football communities. I manually created an Excel file containing all Eredivisie matches from the 2024–2025 season, along with a label indicating whether each match triggered a toxic event on social media.

The main goal of this notebook is to clean, filter, and export the relevant matches that are likely to produce toxicity online. These filtered matches will guide later steps in the project, including tweet scraping, natural language processing (NLP), and social network analysis (SNA).

This step is important because it connects offline events (football matches) to online behavior, allowing for a grounded and event-driven analysis of toxic interactions.


### Import libraries

I start by importing the necessary Python libraries. `pandas` is used to work with tabular data, and `datetime` helps with handling and formatting dates. These libraries are essential for reading the match file, transforming the date column, and preparing the dataset for further analysis.


In [2]:
# ==============================================
# Step 1: Import libraries
# ==============================================

import pandas as pd
from datetime import datetime, timedelta
import pytz

### Load match data

I load the Excel file that contains all Eredivisie matches from the 2024–2025 season into a pandas dataframe. The file includes metadata such as match date, home and away clubs, and a manually assigned label (`toxic_event`) indicating whether a match triggered toxic behavior online. Displaying the dataframe helps confirm that the data is loaded correctly.

In [3]:
# ==============================================
# Step 2: Load matchweek Excel file
# ==============================================

# Path to the excel file
file_path = r"C:\Master\Master project\Matchweeks_Eredivisie V2.xlsx"

# Load the Excel sheet
df = pd.read_excel(file_path)

# Show the first few rows
print(df.head())

   Round              Date       Home Team     Away Team   Time  \
0      1   9 augustus 2024    FC Groningen     NAC Breda  20:00   
1      1  10 augustus 2024       Feyenoord     Willem II  16:30   
2      1  10 augustus 2024             NEC     FC Twente  18:45   
3      1  10 augustus 2024  Almere City FC            AZ  20:00   
4      1  10 augustus 2024             PSV  RKC Waalwijk  21:00   

  Toxicity Trigger Trigger Type(s) Red card Controversie Player error  
0              Yes             NaN      Yes          Yes          NaN  
1              NaN             NaN      NaN          NaN          NaN  
2              NaN             NaN      NaN          NaN          NaN  
3              Yes             NaN      Yes          NaN          NaN  
4              NaN             NaN      NaN          NaN          NaN  


### Filter toxic matches

I start by filling any missing values in the `Toxicity Trigger` column with `"No"`, to avoid issues when filtering. Then I filter the dataframe to keep only the matches where `Toxicity Trigger` is marked as `"Yes"` (case-insensitive). These are the matches that were manually identified as having triggered toxic behavior online. 

I also print the number of toxic matches to confirm the filtering worked correctly, and display them to inspect the results. This subset will be used as the basis for later tweet scraping and analysis.

In [4]:
# ==============================================
# Step 3: Filter toxic matches
# ==============================================

# Fill NaN values in 'Toxicity Trigger' with 'No'
df['Toxicity Trigger'] = df['Toxicity Trigger'].fillna('No')

# Then filter the toxic matches
toxic_matches = df[df['Toxicity Trigger'].str.lower() == 'yes'].copy()

# Confirm count
print(f"Number of toxic matches: {len(toxic_matches)}")

# Preview them
toxic_matches

Number of toxic matches: 79


,Round,Date,Home Team,Away Team,Time,Toxicity Trigger,Trigger Type(s),Red card,Controversie,Player error
0,1,9 augustus 2024,FC Groningen,NAC Breda,20:00,Yes,NaN,Yes,Yes,NaN
3,1,10 augustus 2024,Almere City FC,AZ,20:00,Yes,NaN,Yes,NaN,NaN
13,2,17 augustus 2024,RKC Waalwijk,FC Groningen,21:00,Yes,NaN,Yes,NaN,NaN
18,3,24 augustus 2024,NEC,PEC Zwolle,18:45,Yes,NaN,Yes,NaN,NaN
20,3,24 augustus 2024,NAC Breda,FC Utrecht,21:00,Yes,NaN,Yes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
240,27,30 maart 2025,PSV,Ajax,14:30,Yes,Derby/Rivalry Match,NaN,Yes,NaN
241,27,30 maart 2025,Heracles Almelo,FC Twente,16:45,Yes,Derby/Rivalry Match,NaN,NaN,NaN
244,28,5 april 2025,AZ,Feyenoord,16:30,Yes,Derby/Rivalry Match,NaN,NaN,NaN
250,28,6 april 2025,Sparta,NEC,14:30,Yes,NaN,Yes,NaN,NaN


### Fill missing event-related values

I fill any missing values in the event-related columns: `Trigger Type(s)`, `Red card`, `Controversie`, and `Player error`. These columns describe potential reasons why a match may have triggered toxicity, and missing values could lead to misinterpretation or errors later in the analysis.

By replacing NaNs with `"No"`, I ensure that all toxic matches have complete and consistent metadata for these trigger types.

In [ ]:
# ==============================================
# Step 4: Clean event-related columns
# ==============================================

# Fill missing values for all event-related columns
event_columns = ['Trigger Type(s)', 'Red card', 'Controversie', 'Player error']
toxic_matches[event_columns] = toxic_matches[event_columns].fillna('No')


### Add Twitter handles to teams

I create a dictionary that maps each Eredivisie club to its official Twitter handle. This will be useful later when scraping tweets, as it allows me to search for posts from specific team accounts.

I then add two new columns to the `toxic_matches` dataframe: `Home Handle` and `Away Handle`, which store the Twitter handles for each match’s home and away team.

Finally, I check for any missing values in these new columns. If a team name does not match a key in the dictionary, it will result in a null value. This check helps ensure that all relevant Twitter accounts are correctly linked before moving forward.

In [ ]:
# ==============================================
# Step 5: Map Twitter handles to each team
# ==============================================

# To get the twitter handles of the teams, we can create a dictionary
twitter_handles = {
    "Ajax": "@AFCAjax",
    "Almere City FC": "@AlmereCityFC",
    "AZ": "@AZAlkmaar",
    "Feyenoord": "@Feyenoord",
    "Fortuna Sittard": "@FortunaSittard",
    "Go Ahead Eagles": "@GAEagles",
    "FC Groningen": "@fcgroningen",
    "SC Heerenveen": "@scHeerenveen",
    "Heracles Almelo": "@HeraclesAlmelo",
    "NAC Breda": "@NACnl",
    "NEC": "@necnijmegen",
    "PEC Zwolle": "@PECZwolle",
    "PSV": "@PSV",
    "RKC Waalwijk": "@RKCWAALWIJK",
    "Sparta": "@SpartaRotterdam",
    "FC Twente": "@fctwente",
    "FC Utrecht": "@fcutrecht",
    "Willem II": "@WillemII"
}

# Add Twitter handles to the toxic_matches dataframe
toxic_matches['Home Handle'] = toxic_matches['Home Team'].map(twitter_handles)
toxic_matches['Away Handle'] = toxic_matches['Away Team'].map(twitter_handles)

# Check if any are missing (should return an empty DataFrame ideally)
missing = toxic_matches[toxic_matches['Home Handle'].isnull() | toxic_matches['Away Handle'].isnull()]
print(missing[['Home Team', 'Away Team']])

Empty DataFrame
Columns: [Home Team, Away Team]
Index: []


### Define scraping time windows

To accurately scrape tweets related to each match, I calculate a custom time window around the match date and kickoff time. Since the date column contains Dutch month names, I first define a helper function to translate them into English, making them compatible with Python’s datetime parser.

Next, I define a function that:
- Parses the combined match date and time
- Localizes it to the Europe/Amsterdam timezone
- Converts it to UTC (which is required for tweet scraping via Nitter)

The scraping window starts 24 hours before kickoff and ends 12 hours after. This captures the full conversation cycle around a match — including build-up, live commentary, and post-match reactions. I apply this function to each row and store the start and end timestamps in two new columns.


In [ ]:
# ==============================================
# Step 6: Compute scraping windows in UTC
# ==============================================

# Translate Dutch months to English for parsing
def translate_dutch_date(date_str):
    months = {
        "januari": "January", "februari": "February", "maart": "March", "april": "April",
        "mei": "May", "juni": "June", "juli": "July", "augustus": "August",
        "september": "September", "oktober": "October", "november": "November", "december": "December"
    }
    for dutch, english in months.items():
        if dutch in date_str.lower():
            return date_str.lower().replace(dutch, english)
    return date_str

# Function to calculate scraping window
def compute_time_windows(row):
    # Convert Dutch date
    date_str = translate_dutch_date(str(row['Date']))
    time_str = row['Time']
    
    # Parse and localize to Amsterdam time
    dt_naive = datetime.strptime(f"{date_str} {time_str}", "%d %B %Y %H:%M")
    dt_local = pytz.timezone("Europe/Amsterdam").localize(dt_naive)

    # Convert to UTC
    start_utc = (dt_local - timedelta(hours=24)).astimezone(pytz.utc)
    end_utc = (dt_local + timedelta(hours=12)).astimezone(pytz.utc)

    return pd.Series([start_utc.isoformat(), end_utc.isoformat()])

# Apply to toxic_matches
toxic_matches[["Scrape Start UTC", "Scrape End UTC"]] = toxic_matches.apply(compute_time_windows, axis=1)


### Finalize and export toxic match metadata

I assign a unique match ID to each row using the format "M001", "M002", etc., which provides an easy reference in later stages like scraping or network analysis.

Then I select only the relevant columns that will be needed downstream. These include team names, Twitter handles, trigger types, and the calculated scraping window. By filtering to just the essential information, I simplify the dataset and reduce complexity in later notebooks.

Next, I rename the columns to use lowercase and underscores to make them more code- and scraper-friendly.

Finally, I export the cleaned dataframe to a CSV file at the same location as the original Excel file. This file will be used by the tweet scraper to guide which matches to target and what time windows to use. Printing the save path confirms that the export was successful.


In [ ]:
# ==============================================
# Step 7: Finalize and save cleaned metadata
# ==============================================

# Add a Match ID
toxic_matches["Match ID"] = [f"M{i+1:03}" for i in range(len(toxic_matches))]

# Select only the useful columns
final_columns = [
    "Match ID", "Round", "Date", "Time",
    "Home Team", "Away Team", "Home Handle", "Away Handle",
    "Trigger Type(s)", "Red card", "Controversie", "Player error",
    "Scrape Start UTC", "Scrape End UTC"
]
final_df = toxic_matches[final_columns]

# Rename the columns to scraper-friendly format
new_column_names = {
    'Match ID': 'match_id',
    'Round': 'round',
    'Date': 'date',
    'Time': 'time',
    'Home Team': 'home_team',
    'Away Team': 'away_team',
    'Home Handle': 'home_handle',
    'Away Handle': 'away_handle',
    'Trigger Type(s)': 'trigger_types',
    'Red card': 'red_card',
    'Controversie': 'controversie',
    'Player error': 'player_error',
    'Scrape Start UTC': 'scrape_start_utc',
    'Scrape End UTC': 'scrape_end_utc'
}
final_df.rename(columns=new_column_names, inplace=True)

# Set the same path as the Excel file
save_path = r"C:\Master\Master project\toxic_match_metadata_cleaned.csv"

# Save the cleaned and enriched toxic match metadata
final_df.to_csv(save_path, index=False)

print(f"File saved to: {save_path}")

File saved to: C:\Master\Master project\toxic_match_metadata_cleaned.csv


C:\Users\timzi\AppData\Local\Temp\ipykernel_95368\3804462872.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.rename(columns=new_column_names, inplace=True)
